In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

### data

In [2]:
data = input_data.read_data_sets("D:/work/data/Python/tensorflow/mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/work/data/Python/tensorflow/mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/work/data/Python/tensorflow/mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/work/data/Python/tensorflow/mnist/data/t10k-images-idx3-ubyte.gz
Extracting D:/work/data/Python/tensorflow/mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 参数

In [3]:
learning_rate = 0.01
epochs = 100
batch_size = 64

n_features = 784
n_classes = 10
n_hidden = 128

### model

In [4]:
g = tf.Graph()
with g.as_default():
    X = tf.placeholder(tf.float32, [None, 784], name="X")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y")
    
    weights = {
        "W1": tf.Variable(tf.truncated_normal([784, n_hidden])),
        "out": tf.Variable(tf.truncated_normal([n_hidden, n_classes]))
    }
    
    bias = {
        "b1": tf.Variable(tf.zeros([n_hidden])),
        "out": tf.Variable(tf.zeros([n_classes]))
    }
    
    fc1 = tf.matmul(X, weights["W1"]) + bias["b1"]
    fc1 = tf.nn.relu(fc1)
    out = tf.matmul(fc1, weights["out"]) + bias["out"]
    
    pred = tf.nn.softmax(out, name="prediction")
    
    # loss
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=pred))
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    
    # train op
    train_op = optimizer.minimize(cost)
    
    # accuracy
    pred_labels = tf.argmax(pred, axis=1, name="prediction_label")
    correct = tf.equal(tf.argmax(Y, axis=1), pred_labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

### train

In [5]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = data.train.num_examples // batch_size
        
        for i in range(total_batch):
            batch_x, batch_y = data.train.next_batch(batch_size=batch_size)
            _ = sess.run([train_op], feed_dict={X: batch_x, Y: batch_y})
        
        if (epoch + 1) % 20 == 0:
            train_acc, c = sess.run([accuracy, cost], feed_dict={X: data.train.images[:5000], 
                                                          Y: data.train.labels[:5000]})
            valid_acc = sess.run(accuracy, feed_dict={X: data.validation.images, Y: data.validation.labels})

            print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, c), end="")
            print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))

    test_acc = sess.run(accuracy, feed_dict={X: data.test.images, Y: data.test.labels})
    print('Test ACC: %.3f' % test_acc)

Epoch: 020 | AvgCost: 2.084 | Train/Valid ACC: 0.376/0.367
Epoch: 040 | AvgCost: 2.039 | Train/Valid ACC: 0.421/0.415
Epoch: 060 | AvgCost: 2.029 | Train/Valid ACC: 0.431/0.438
Epoch: 080 | AvgCost: 2.003 | Train/Valid ACC: 0.457/0.448
Epoch: 100 | AvgCost: 1.993 | Train/Valid ACC: 0.467/0.490
Test ACC: 0.484
